In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
from src.heikin_ashi import heikin_ashi
from src.indicators import supertrend, VWAP2, smc, wavetrend3d, atr
import pandas_ta as ta

In [3]:
auxusd = pd.read_csv("./data_directory/AUXUSD.csv")
auxusd['timestamp'] = pd.to_datetime(auxusd['timestamp'], unit='ms')
auxusd['timestamp'] = pd.to_datetime(auxusd['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
auxusd.set_index('timestamp', inplace=True)
auxusd.drop_duplicates(inplace=True)

In [4]:
eurgbp = pd.read_csv("./data_directory/EURGBP.csv")
eurgbp['timestamp'] = pd.to_datetime(eurgbp['timestamp'], unit='ms')
eurgbp['timestamp'] = pd.to_datetime(eurgbp['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
eurgbp.set_index('timestamp', inplace=True)
eurgbp.drop_duplicates(inplace=True)

In [5]:
forex_data = pd.read_csv("./data_directory/EURUSD_2021_2023.csv")
forex_data['timestamp'] = pd.to_datetime(forex_data['timestamp'], unit='ms')
forex_data['timestamp'] = pd.to_datetime(forex_data['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
forex_data.set_index('timestamp', inplace=True)
forex_data.drop_duplicates(inplace=True)

In [6]:
eurusd = pd.read_csv("./data_directory/EURUSD_NEW.csv")
eurusd.set_index("Date", inplace=True)
eurusd.index = pd.to_datetime(eurusd.index)
eurusd.drop_duplicates(inplace=True)

In [ ]:
# Der choCH wird hier programmiert

def choCH(data, length, band):
    chocolad = pd.Series(index=data.index)
    ob = smc(data, length, band, "30T")
    prevvalue = 0
    orderblocks = []
    i = 0
    if(band == 0):
        for index, values in ob.iteritems():
            if(values > 0 and values != prevvalue):
                prevvalue = values
                give = [i, values]
                orderblocks.append(give)
            i += 1
        for i, (candle, price) in enumerate(orderblocks):
            if price > orderblocks[i-2][1] and orderblocks[i-1][1] < orderblocks[i-2][1] and orderblocks[i-2][1] < orderblocks[i-3][1] and orderblocks[i-3][1] < orderblocks[i-4][1]:
                chocolad.iloc[candle] = price
    elif(band == 1):
        for index, values in ob.iteritems():
            if(values > 0 and values != prevvalue):
                prevvalue = values
                give = [i, values]
                orderblocks.append(give)
            i += 1
        for i, (candle, price) in enumerate(orderblocks):
            if price < orderblocks[i-2][1] and orderblocks[i-1][1] > orderblocks[i-2][1] and orderblocks[i-2][1] > orderblocks[i-3][1] and orderblocks[i-3][1] > orderblocks[i-4][1]:
                chocolad.iloc[candle] = price
    return chocolad
    

hello = choCH(forex_data, 10, 0)




In [ ]:

for index, value in hello.iteritems():
    if value > 0:
        print(index)

In [8]:
lower = supertrend(forex_data, 10, 2.5, 0)

In [10]:
lower[:30]

timestamp
2021-06-20 19:00:00         NaN
2021-06-20 19:09:00    1.186345
2021-06-20 19:36:00    1.186086
2021-06-20 19:42:00    1.186086
2021-06-20 19:45:00    1.186033
2021-06-20 19:51:00    1.186033
2021-06-20 19:54:00    1.186008
2021-06-20 20:00:00    1.185840
2021-06-20 20:03:00    1.185840
2021-06-20 20:06:00    1.185840
2021-06-20 20:12:00    1.185840
2021-06-20 20:15:00    1.185755
2021-06-20 20:18:00    1.185696
2021-06-20 20:24:00    1.185673
2021-06-20 20:27:00         NaN
2021-06-20 20:30:00         NaN
2021-06-20 20:33:00         NaN
2021-06-20 20:36:00         NaN
2021-06-20 20:42:00         NaN
2021-06-20 20:45:00         NaN
2021-06-20 20:48:00         NaN
2021-06-20 20:51:00         NaN
2021-06-20 20:57:00         NaN
2021-06-20 21:00:00         NaN
2021-06-20 21:03:00         NaN
2021-06-20 21:06:00         NaN
2021-06-20 21:12:00         NaN
2021-06-20 21:15:00         NaN
2021-06-20 21:18:00         NaN
2021-06-20 21:21:00         NaN
Name: dt, dtype: float64

In [21]:
data = forex_data
lookback=10 
multiplier=2.5
band=0

high = data["High"]
low = data["Low"]
close = data["Close"]
# ATR

tr1 = pd.DataFrame(high - low)
tr2 = pd.DataFrame(abs(high - close.shift(1)))
tr3 = pd.DataFrame(abs(low - close.shift(1)))
frames = [tr1, tr2, tr3]
tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
atr = tr.ewm(lookback).mean()

# H/L AVG AND BASIC UPPER & LOWER BAND

hl_avg = (high + low) / 2
upper_band = (hl_avg + multiplier * atr).fillna(method="ffill").dropna()
lower_band = (hl_avg - multiplier * atr).dropna()

# FINAL UPPER BAND
final_bands = pd.DataFrame(columns = ['upper', 'lower'], index=upper_band.index)
final_bands["upper"] = 0.0
final_bands["lower"] = 0.0



                     upper  lower
timestamp                        
2021-06-20 19:00:00    0.0    0.0
2021-06-20 19:09:00    0.0    0.0
2021-06-20 19:36:00    0.0    0.0
2021-06-20 19:42:00    0.0    0.0
2021-06-20 19:45:00    0.0    0.0
...                    ...    ...
2023-05-19 20:51:00    0.0    0.0
2023-05-19 20:54:00    0.0    0.0
2023-05-19 20:57:00    0.0    0.0
2023-05-19 21:00:00    0.0    0.0
2023-05-19 21:03:00    0.0    0.0

[238830 rows x 2 columns]


In [ ]:
def supertrend(data, lookback=10, multiplier=2.5, band=0):
    high = data["High"]
    low = data["Low"]
    close = data["Close"]
    # ATR
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.ewm(lookback).mean()
    
    # H/L AVG AND BASIC UPPER & LOWER BAND
    
    hl_avg = (high + low) / 2
    upper_band = (hl_avg + multiplier * atr).fillna(method="ffill").dropna()
    lower_band = (hl_avg - multiplier * atr).dropna()
    
    # FINAL UPPER BAND
    final_bands = pd.DataFrame(columns = ['upper', 'lower'])
    final_bands.iloc[:,0] = [x for x in upper_band - upper_band]
    final_bands.iloc[:,1] = final_bands.iloc[:,0]
    for i in range(len(final_bands)):
        if i == 0:
            final_bands.iloc[i,0] = 0
        else:
            if (upper_band[i] < final_bands.iloc[i-1,0]) | (close[i-1] > final_bands.iloc[i-1,0]):
                final_bands.iloc[i,0] = upper_band[i]
            else:
                final_bands.iloc[i,0] = final_bands.iloc[i-1,0]
    
    # FINAL LOWER BAND
    
    for i in range(len(final_bands)):
        if i == 0:
            final_bands.iloc[i, 1] = 0
        else:
            if (lower_band[i] > final_bands.iloc[i-1,1]) | (close[i-1] < final_bands.iloc[i-1,1]):
                final_bands.iloc[i,1] = lower_band[i]
            else:
                final_bands.iloc[i,1] = final_bands.iloc[i-1,1]
    
    # SUPERTREND
    
    supertrend = pd.DataFrame(index=upper_band.index, columns=[f'supertrend_{lookback}'])
    supertrend.iloc[:, 0] = upper_band.values
    supertrend.iloc[:,0] = [x for x in final_bands['upper'] - final_bands['upper']]
    
    for i in range(1, len(supertrend)):
        if i == 0:
            supertrend.iloc[i, 0] = 0
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and close[i] < final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and close[i] > final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and close[i] > final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and close[i] < final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
    
    supertrend = supertrend.set_index(upper_band.index)
    supertrend = supertrend.dropna()[1:]
    
    # ST UPTREND/DOWNTREND
    
    upt = []
    dt = []
    close = close.iloc[len(close) - len(supertrend):]

    for i in range(len(supertrend)):
        if close[i] > supertrend.iloc[i, 0]:
            upt.append(supertrend.iloc[i, 0])
            dt.append(np.nan)
        elif close[i] < supertrend.iloc[i, 0]:
            upt.append(np.nan)
            dt.append(supertrend.iloc[i, 0])
        else:
            upt.append(np.nan)
            dt.append(np.nan)
    new_index = pd.date_range(start= data.index[0] , periods=1, freq='3min')
    st, upt, dt = pd.Series(supertrend.iloc[:, 0]), pd.Series(upt), pd.Series(dt)
    upt.index, dt.index = supertrend.index, supertrend.index
    upt = pd.concat([pd.Series([np.nan], index=new_index), upt])
    dt = pd.concat([pd.Series([np.nan], index=new_index), dt])
    data["upt"] = upt
    data["upt"].index = data.index
    data["dt"] = dt
    data["dt"].index = data.index
    if(band==0):
        return data["dt"]
    elif(band==1):
        return data["upt"]
    
def swings(data, length, timeframe):
    resampled = data.resample(timeframe).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
    upper = resampled['High'].rolling(window=length).max()
    lower = resampled['Low'].rolling(window=length).min()

    os = np.where(resampled['High'].shift(length) > upper, 0, np.where(resampled['Low'].shift(length) < lower, 1, np.nan))
    os = pd.Series(os).fillna(method='ffill')
    top = np.where((os == 0) & (os.shift() != 0), resampled['High'].shift(length), 0)
    btm = np.where((os == 1) & (os.shift() != 1), resampled['Low'].shift(length), 0)
    series1 = pd.Series(top).shift(-length)
    series2 = pd.Series(btm).shift(-length)
    series1.index = resampled.index
    series2.index = resampled.index
    return [series1, series2]